In [1]:
import os
import sys
from pathlib import Path

# Add the project root to the Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

from dotenv import load_dotenv
load_dotenv()
from langchain_upstage import UpstageEmbeddings
embeddings = UpstageEmbeddings(model="embedding-passage")

DB_URI = os.environ["POSTGRES_URI"]     
# from langchain_postgres import PGEngine, PGVectorStore
# pg_engine = PGEngine.from_connection_string(
#     url=DB_URI
# )

from langchain_postgres import PGVector
from langchain_core.documents import Document
vector_store = PGVector(
    embeddings=embeddings,
    collection_name="SPRI_ALL",
    connection=DB_URI
)

In [2]:
import pickle
papers = []
for path in sorted(Path(project_root).joinpath("outputs").glob("*_split_documents.pkl")):
    
    with open(path, "rb") as f:
        split_documents = pickle.load(f)
        for doc in split_documents:
            doc.metadata["title"] = path.stem.replace("_output_split_documents", "")
    papers.append(split_documents)


In [3]:
len(papers)

3

In [4]:
papers[1][0].metadata

{'page': 1,
 'image_id': [],
 'image_path': [],
 'text_summary': [],
 'image_summary': [],
 'id': 0,
 'title': 'SPRI_2023'}

In [5]:
retriever = vector_store.as_retriever(search_kwargs={"k": 10, "filter": 
{"$and": [
            {"title": {"$eq": "SPRI_2023"}},
                        
        ]}
})

In [6]:
retriever.invoke("AI 트렌드")

[Document(id='cbeef5ca-16a7-494b-a0ab-73468310d1f8', metadata={'id': 95, 'page': 54, 'title': 'SPRI_2023', 'image_id': [], 'image_path': [], 'text_summary': '요약: SPRi 이슈리포트 IS-159의 AI Index 2023 주요 내용과 시사점\n\n- 정부 전략과 이행 가속화\n  - 국가초고성능컴퓨팅 혁신전략(‘21.5.28), 국산 AI반도체를 활용한 K-클라우드 추진방안(‘22.12.12), 초거대 AI 경쟁력 강화방안(‘23.4.14) 등 정부 추진 전략의 단계적 이행 가속 필요.\n\n- AI 윤리와 거버넌스\n  - AI 오용 사례 증가와 생성 AI 확대로 AI 윤리에 대한 관심 증가.\n  - 공정성·편향 등 신뢰성 검증을 위한 구체적 기준과 가이드라인 필요.\n  - 활용 분야별 주요 지표 확립이 중요하며, 학습 데이터와 완화 방법에 따라 편향/독성의 영향이 달라질 수 있음.\n  - 편향성 및 독성 완화를 위한 명령 조정, 데이터 필터링 등 가이드라인 제시 필요.\n\n- 경제·투자 흐름\n  - 세계적 경기 침체의 영향으로 AI 분야 투자와 도입이 감소하는 가운데, 한국은 투자 확대를 통해 6위권 유지.\n  - 전 세계 투자 규모·건수·피투자기업 수가 감소하는 추세.\n  - 대형(10억 달러 이상) 투자를 제외한 규모별 투자 건수 감소로 신규 피투자 기업 수가 줄고 투자 집중화 심화.\n  - 미국의 영향력 여전히 높아 글로벌 AI 스타트업 투자의 약 52%를 차지하고 AI 채용 공고 비중도 높음.\n  - 중국은 AI 민간투자 규모와 AI 스타트업 수 등에서 2위 유지; 영국, 캐나다, 이스라엘, 독일 등도 상위 시장에 지속 포진.\n  - 한국은 근로자 AI 역량과 산업용 로봇 도입이 높고, 최근 AI 스타트업 관련 흐름도 주목.\n\n- 종합 시사점\n  - 정부의 전략 이행 가속과 함께 윤리 가이드라인 및 데

In [7]:
from langchain_community.retrievers import BM25Retriever

In [9]:
docs = []
for paper in papers:
    docs.extend(paper)

In [10]:
len(docs)

247

In [ ]:
from sqlalchemy import create_engine, text

engine = create_engine(DB_URI)
def sql_query() -> list[tuple[str, dict]]:
    with engine.connect() as conn:
        # Custom SQL query
        query = conn.execute(text(f"""
            SELECT 
                e.document,
                e.cmetadata            
            FROM langchain_pg_embedding e
            JOIN langchain_pg_collection c ON e.collection_id = c.uuid
            WHERE c.name = :collection_name
                AND e.cmetadata->>'title' = :title
        """), {
            "collection_name": "SPRI_ALL",
            "title": "SPRI_2023",
        })
        
        return query.fetchall()

In [35]:
arr = []
for r in result:
    arr.append(Document(page_content=r[0], metadata=r[1]))

In [38]:
arr =sorted(arr, key=lambda x: x.metadata["id"])

In [41]:
len(arr)

108

In [42]:
bm25_retriever = BM25Retriever.from_documents(arr)
bm25_retriever.k = 10

In [43]:
bm25_retriever.invoke("AI 트렌드")

[Document(metadata={'id': 38, 'page': 23, 'title': 'SPRI_2023', 'image_id': [265, 269], 'image_path': ['/images/SPRI_2023_cropped_chart_265.png', '/images/SPRI_2023_cropped_table_269.png'], 'text_summary': '다음 내용을 한국어로 요약합니다.\n\n- 이 글은 SPRi 이슈리포트 IS-159로, AI Index 2023의 주요 내용과 시사점을 다룹니다.\n- AI 사고 및 논쟁 수의 증가 추세를 표로 제시하며, 2012–2021년에 걸친 증가를 보여줌. 2022년 보고된 사건은 심사 과정으로 인해 미포함되며, 출처는 Stanford HAI의 AI Index Report 2023.\n- 2022년 보고된 대표 사건 다섯 가지 요약:\n  1) 볼로디미르 젤렌스키 대통령의 항복 영상이 딥페이크로 확산되어 우크라이나 전쟁 맥락을 조작하려는 시도.\n  2) Verus가 교도소 수감자 전화통화를 AI 음성인식으로 모니터링, 프라이버시 및 차별 우려 제기.\n  3) 인텔의 학생 감정 모니터링 시스템 개발(패턴인식 기반)으로 Zoom 등에서 학생의 감정을 식별하는 기술 도입 시 프라이버시 및 차별 문제 제기.\n  4) 런던 경찰청의 갱 폭력 매트릭스 개발로 1,000명 이상 데이터를 활용해 위험성을 측정하나 정확성 저하와 특정 소수 민족 차별 가능성 제기.\n  5) Midjourney의 텍스트->이미지 생성 서비스 등 생성AI의 이용으로 저작권, 데이터 학습, 고용 대체, 개인정보보호 이슈 제기.\n- 이 사례들은 프라이버시, 차별, 저작권, 고용 영향 등 AI 활용에 따른 법적·윤리적 이슈가 광범위하게 제기되고 있음을 보여주며, AI 거버넌스의 필요성을 시사합니다.', 'image_summary': ['다음 이미지는 연도별 AI 사고 및 논쟁 수의 변화를 보여주는 바 차트와 2022년 대표 사건 사례를 나

In [13]:
retriever.invoke("AI 트렌드")

[Document(id='cbeef5ca-16a7-494b-a0ab-73468310d1f8', metadata={'id': 95, 'page': 54, 'title': 'SPRI_2023', 'image_id': [], 'image_path': [], 'text_summary': '요약: SPRi 이슈리포트 IS-159의 AI Index 2023 주요 내용과 시사점\n\n- 정부 전략과 이행 가속화\n  - 국가초고성능컴퓨팅 혁신전략(‘21.5.28), 국산 AI반도체를 활용한 K-클라우드 추진방안(‘22.12.12), 초거대 AI 경쟁력 강화방안(‘23.4.14) 등 정부 추진 전략의 단계적 이행 가속 필요.\n\n- AI 윤리와 거버넌스\n  - AI 오용 사례 증가와 생성 AI 확대로 AI 윤리에 대한 관심 증가.\n  - 공정성·편향 등 신뢰성 검증을 위한 구체적 기준과 가이드라인 필요.\n  - 활용 분야별 주요 지표 확립이 중요하며, 학습 데이터와 완화 방법에 따라 편향/독성의 영향이 달라질 수 있음.\n  - 편향성 및 독성 완화를 위한 명령 조정, 데이터 필터링 등 가이드라인 제시 필요.\n\n- 경제·투자 흐름\n  - 세계적 경기 침체의 영향으로 AI 분야 투자와 도입이 감소하는 가운데, 한국은 투자 확대를 통해 6위권 유지.\n  - 전 세계 투자 규모·건수·피투자기업 수가 감소하는 추세.\n  - 대형(10억 달러 이상) 투자를 제외한 규모별 투자 건수 감소로 신규 피투자 기업 수가 줄고 투자 집중화 심화.\n  - 미국의 영향력 여전히 높아 글로벌 AI 스타트업 투자의 약 52%를 차지하고 AI 채용 공고 비중도 높음.\n  - 중국은 AI 민간투자 규모와 AI 스타트업 수 등에서 2위 유지; 영국, 캐나다, 이스라엘, 독일 등도 상위 시장에 지속 포진.\n  - 한국은 근로자 AI 역량과 산업용 로봇 도입이 높고, 최근 AI 스타트업 관련 흐름도 주목.\n\n- 종합 시사점\n  - 정부의 전략 이행 가속과 함께 윤리 가이드라인 및 데